# Sozialplan - Abbau von Arbeitszeitvolumen

In [12]:
import csv
# Initialisieren der Klasse:
class Mitarbeiter:
    def __init__(self, vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert, arbeitsvertrag):
        self.vorname = vorname
        self.nachname = nachname
        self.alter = alter
        self.betriebszugehoerigkeit = betriebszugehoerigkeit
        self.ehepartner_unterhalt = ehepartner_unterhalt
        self.kinder_unterhalt = kinder_unterhalt
        self.schwerbehindert = schwerbehindert
        self.arbeitsvertrag = arbeitsvertrag
        self.punkte = alter + 1.5 * betriebszugehoerigkeit + 5 * ehepartner_unterhalt + 5 * kinder_unterhalt + 8 * schwerbehindert
        self.dismiss = False
        self.gruppe = None
        
  # Für den Fall, dass das Geburtsdatum und nicht das Alter in den Daten steht        
def berechne_alter(geburtsdatum):
        heute = datetime.date.today()  # Heutiges Datum
        geburtsdatum = datetime.datetime.strptime(geburtsdatum, '%Y-%m-%d').date()  # Geburtsdatum in Date-Format umwandeln
        alter = heute.year - geburtsdatum.year - ((heute.month, heute.day) < (geburtsdatum.month, geburtsdatum.day))
        return alter      
        
        
#Einlesen der Daten
def read_data(filename):
    mitarbeiter_list = []
    with open(filename, 'r', newline='') as file:
        reader = csv.reader(file, delimiter=',') # Delimiter ggf anpassen!
        fieldnames = next(reader)
        fieldnames = [field.strip().lower() for field in fieldnames]
        reader = csv.DictReader(file, fieldnames=fieldnames, delimiter=',')
        for row in reader:
            vorname = row['vorname'].strip()
            nachname = row['nachname'].strip()
            name = f"{vorname} {nachname}"
            
             # Berechnen des Alters basierend auf dem Geburtsdatum oder direkt verwenden, falls vorhanden
            if 'geburtsdatum' in row and row['geburtsdatum'].strip():
                geburtsdatum = row['geburtsdatum'].strip()
                alter = berechne_alter(geburtsdatum)
            elif 'alter' in row and row['alter'].strip():
                alter = int(row['alter'].strip())
            else:
                raise ValueError("Weder Alter noch Geburtsdatum angegeben")
            
            betriebszugehoerigkeit = int(row['betriebszugehoerigkeit'].strip())
            ehepartner_unterhalt = int(row['ehepartnerunterhalt'].strip())
            kinder_unterhalt = int(row['kinderunterhalt'].strip())
            schwerbehindert = int(row['schwerbehindert'].strip())
            arbeitsvertrag = int(row['arbeitsvertrag'].strip())
            mitarbeiter = Mitarbeiter(vorname, nachname, alter, betriebszugehoerigkeit, ehepartner_unterhalt, kinder_unterhalt, schwerbehindert, arbeitsvertrag)
            mitarbeiter_list.append(mitarbeiter)
    return mitarbeiter_list

# Gruppeneinteilung nach Alter
def assign_groups(mitarbeiter_list):
    groups = {1: [], 2: [], 3: [], 4: []}
    for mitarbeiter in mitarbeiter_list:
        if mitarbeiter.alter <= 30:
            group = 1
        elif 31 <= mitarbeiter.alter <= 40:
            group = 2
        elif 41 <= mitarbeiter.alter <= 50:
            group = 3
        else:
            group = 4
        mitarbeiter.gruppe = group
        groups[group].append(mitarbeiter)
    return groups

#Entlassungslogik
def mark_for_dismissal_by_hours(groups, target_hours):
    remaining_hours = target_hours
    sorted_groups = {group: sorted(mitarbeiter_list, key=lambda x: x.punkte) for group, mitarbeiter_list in groups.items()}
    
    total_groups = len(sorted_groups)
    group_indices = {group: 0 for group in sorted_groups}  # Initiale Indizes für jede Gruppe
    all_groups_done = False

    while not all_groups_done and remaining_hours > 0:
        all_groups_done = True
        for group in sorted_groups:
            if group_indices[group] < len(sorted_groups[group]):
                mitarbeiter = sorted_groups[group][group_indices[group]]
                if remaining_hours - mitarbeiter.arbeitsvertrag >= 0:
                    mitarbeiter.dismiss = True
                    remaining_hours -= mitarbeiter.arbeitsvertrag
                    group_indices[group] += 1
                else:
                    remaining_hours = -1  # Setzt remaining_hours auf -1, um zu signalisieren, dass das Limit erreicht ist
                    break
                all_groups_done = False  # Setzt all_groups_done auf False, wenn mindestens eine Gruppe noch nicht fertig ist

def print_results(groups):
    for group, mitarbeiter_list in groups.items():
        sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: x.punkte)
        print(f"Gruppe {group}:")
        for mitarbeiter in sorted_mitarbeiter:
            if mitarbeiter.dismiss:
                print(f"{mitarbeiter.vorname} {mitarbeiter.nachname} (Punkte: {mitarbeiter.punkte}, Arbeitsvertrag: {mitarbeiter.arbeitsvertrag}) - Entlassung empfohlen")
            else:
                print(f"{mitarbeiter.vorname} {mitarbeiter.nachname} (Punkte: {mitarbeiter.punkte}, Arbeitsvertrag: {mitarbeiter.arbeitsvertrag})")

def save_results_to_csv(groups, filename='results.csv'):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Gruppe', 'Vorname', 'Nachname', 'Alter', 'Betriebszugehoerigkeit', 'Ehepartner_Unterhalt', 
                      'Kinder_Unterhalt', 'schwerbehindert', 'Punkte', 'Empfehlung']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for group, mitarbeiter_list in sorted(groups.items()):
            sorted_mitarbeiter = sorted(mitarbeiter_list, key=lambda x: x.punkte)
            for mitarbeiter in sorted_mitarbeiter:
                writer.writerow({
                    'Gruppe': group,
                    'Vorname': mitarbeiter.vorname,
                    'Nachname': mitarbeiter.nachname,
                    'Alter': mitarbeiter.alter,
                    'Betriebszugehoerigkeit': mitarbeiter.betriebszugehoerigkeit,
                    'Ehepartner_Unterhalt': 'ja' if mitarbeiter.ehepartner_unterhalt == 1
                                                 else '',
                    'Kinder_Unterhalt': mitarbeiter.kinder_unterhalt,
                    'schwerbehindert': 'ja' if mitarbeiter.schwerbehindert == 1 
                                            else '',
                    'Punkte': mitarbeiter.punkte,
                    'Empfehlung': 'Entlassungsempfehlung' if mitarbeiter.dismiss 
                                       else ''
                })

def main():
    filename = 'new_names_synthetic_employees.csv'
    target_hours = 1670
    mitarbeiter_list = read_data(filename)
    if len(mitarbeiter_list) > 0:
        groups = assign_groups(mitarbeiter_list)
        mark_for_dismissal_by_hours(groups, target_hours)
        print_results(groups)
        save_results_to_csv(groups, 'Entlassungsempfehlung_nach_Stunden.csv')

if __name__ == "__main__":
    main()


Gruppe 1:
Paul Taylor (Punkte: 34.5, Arbeitsvertrag: 145) - Entlassung empfohlen
Emily Pearson (Punkte: 35.5, Arbeitsvertrag: 80) - Entlassung empfohlen
Robert Martin (Punkte: 36.5, Arbeitsvertrag: 160) - Entlassung empfohlen
Megan Marshall (Punkte: 43.5, Arbeitsvertrag: 167)
Tara Burgess (Punkte: 44.5, Arbeitsvertrag: 80)
Daniel Gallegos (Punkte: 45.0, Arbeitsvertrag: 80)
Leslie Martinez (Punkte: 45.5, Arbeitsvertrag: 167)
Katelyn Snyder (Punkte: 46.5, Arbeitsvertrag: 130)
George Moyer (Punkte: 52.5, Arbeitsvertrag: 160)
Garrett Vasquez (Punkte: 53.5, Arbeitsvertrag: 80)
Cindy Parker (Punkte: 55.0, Arbeitsvertrag: 160)
Zachary Mitchell (Punkte: 57.5, Arbeitsvertrag: 167)
Michelle Hernandez (Punkte: 62.0, Arbeitsvertrag: 80)
Gruppe 2:
Gregory Page (Punkte: 48.5, Arbeitsvertrag: 145) - Entlassung empfohlen
Robin Skinner (Punkte: 54.5, Arbeitsvertrag: 160) - Entlassung empfohlen
Debra Myers (Punkte: 71.0, Arbeitsvertrag: 167) - Entlassung empfohlen
Monica King (Punkte: 79.0, Arbeitsvertr